In [15]:
import pandas as pd
import numpy as np
import math
from warnings import simplefilter

simplefilter(action='ignore') #忽略警告



for level in range(0,3):
    if level == 0 :
        index_max = 199
    if level == 1 :
        index_max = 90
    if level == 2 :
        index_max = 132
    for index in range(1,index_max+1):
        file_name="data_Skip_frame_completion/level_"+str(level)+"/"+str(level)+"_"+str(index)+".csv";
        print(file_name)
        data = pd.read_csv(file_name,encoding = 'gb2312',names=['id', 'frame', 'x', 'y'])  # 读入csv文件中的特征
        #眼部
        #求最大值
        #定义几个要用的变量
        left_max=0;
        right_max=0;
        length=data.shape[0];
        for i in range(0,length-1):
            i=i+1;
            id=data.iloc[i]["id"];
            if id == "39":
                x39=float(data.iloc[i]["x"])
                x36=float(data.iloc[i-3]["x"])
                x45=float(data.iloc[i+6]["x"])
                x42=float(data.iloc[i+3]["x"])
                w_left_sub=x39-x36;#左眼的宽度差值
                w_right_sub=x45-x42;#右眼的宽度差值
                y38=float(data.iloc[i-1]["y"])
                y40=float(data.iloc[i+1]["y"])
                y43=float(data.iloc[i+4]["y"])
                y47=float(data.iloc[i+8]["y"])
                h_left_sub=y40-y38;#左眼的高度差值
                h_right_sub=y47-y43;#右眼的高度差值

                #求最大值
                left_max_temp=w_left_sub*h_left_sub;
                right_max_temp=w_right_sub*h_right_sub;
                if left_max_temp>left_max:
                    left_max=left_max_temp;
                if right_max_temp>right_max:
                    right_max=right_max_temp;

#         print(left_max)                    
        length=data.shape[0];
        xl={};#左眼中间横坐标
        yl={};#左眼中间纵坐标
        xr={};#右眼中间横坐标
        yr={};#右眼中间纵坐标
        xm={};#上唇中间的横坐标
        ym={};#下唇中间横坐标
        A={};
        B={};
        C={};
        d={};#上唇中部到两眼连线的距离
        head_amplitude={}; #头部摆动幅度

        left_max_ratio={};
        right_max_ratio={};
        eyes_ratio={};

        K={};
        MOA={};
        
        for i in range(0,length-1):
            i=i+1;
            id=data.iloc[i]["id"];
            frame=int(data.iloc[i]["frame"]);
            if id == "39" :
                xl[frame]=(float(data.iloc[i-2]["x"])+float(data.iloc[i-1]["x"])+float(data.iloc[i+1]["x"])+float(data.iloc[i+2]["x"]))/4
                yl[frame]=(float(data.iloc[i-2]["y"])+float(data.iloc[i-1]["y"])+float(data.iloc[i+1]["y"])+float(data.iloc[i+2]["y"]))/4
             
                x39=float(data.iloc[i]["x"])
                x36=float(data.iloc[i-3]["x"])
                x45=float(data.iloc[i+6]["x"])
                x42=float(data.iloc[i+3]["x"])
                w_left_sub=x39-x36;#左眼的宽度差值
                w_right_sub=x45-x42;#右眼的宽度差值
                y38=float(data.iloc[i-1]["y"])
                y40=float(data.iloc[i+1]["y"])
                y43=float(data.iloc[i+4]["y"])
                y47=float(data.iloc[i+8]["y"])
                h_left_sub=y40-y38;#左眼的高度差值
                h_right_sub=y47-y43;#右眼的高度差值

                #求百分比
                left_max_temp=w_left_sub*h_left_sub;
                left_max_ratio[frame]=left_max_temp/left_max;#左眼目前/左眼最大
                right_max_temp=w_right_sub*h_right_sub;
                right_max_ratio[frame]=right_max_temp/right_max;
                eyes_ratio[frame]=(left_max_ratio[frame]+right_max_ratio[frame])/2;

            if id == "45" :
                xr[frame]=(float(data.iloc[i-2]["x"])+float(data.iloc[i-1]["x"])+float(data.iloc[i+1]["x"])+float(data.iloc[i+2]["x"]))/4
                yr[frame]=(float(data.iloc[i-2]["y"])+float(data.iloc[i-1]["y"])+float(data.iloc[i+1]["y"])+float(data.iloc[i+2]["y"]))/4        
            if id == "51" :
                xm[frame]=float(data.iloc[i]["x"])
                ym[frame]=float(data.iloc[i]["y"])
            if id == "60" :
                x60=float(data.iloc[i]["x"])
                y60=float(data.iloc[i]["y"])
                x62=float(data.iloc[i+2]["x"])
                y62=float(data.iloc[i+2]["y"])
                x66=float(data.iloc[i+6]["x"])
                y66=float(data.iloc[i+6]["y"])
                #点62到点60距离的平方
                d_62_60_2=pow((x62-x60), 2)+ pow((y62-y60), 2)
                d_66_60_2=pow((x66-x60), 2)+ pow((y66-y60), 2)
                d_66_62_2=pow((x66-x62), 2)+ pow((y66-y62), 2)

                if d_62_60_2 == 0:
                    d_62_60_2 = 1
                if d_66_60_2 == 0:
                    d_66_60_2 = 1
                #距离
                d_62_60=pow(d_62_60_2, 0.5)
                d_66_60=pow(d_66_60_2, 0.5)
                d_66_62=pow(d_66_62_2, 0.5)
                MOA[frame]=(d_62_60_2 + d_66_60_2 - d_66_62_2) / (2 * d_62_60 * d_66_60)
#                 print("MOA是",MOA[frame])

        dmax=0;

   #求上唇中间到双眼连线的距离
        for j in yr:
            A[j]=yr[j]-yl[j];
            B[j]=xl[j]-xr[j];
            C[j]=xr[j]*yl[j]-xl[j]*yr[j]
            d[j] = np.abs(A[j] * xm[j] + B[j] * ym[j] + C[j] ) / (np.sqrt(A[j]**2 + B[j]**2))
            dmax=max(dmax,d[j]);#更新最大值

#         print("最大的d是：",dmax)

        #求出头的摆动幅度（转化为度）
        for k in d:
            head_amplitude[k]=math.degrees(math.acos(d[k]/dmax));

            #输出到文件

        #字典中的key值即为csv中的列名

        frame_col={};

#         for k in range(1,frame+1):
#             frame_col[k]=k;
            
        for k in yr:
            frame_col[k]=k;

        df = pd.DataFrame({'frame':frame_col,'head_amplitude':head_amplitude,
                          'eyes_ratio':eyes_ratio,
                          'MOA':MOA})
 
        file_name_new="data_new/level_"+str(level)+"/"+str(level)+"_"+str(index)+".csv";
        df.to_csv(file_name_new,index=False,sep=',')

data_Skip_frame_completion/level_1/1_1.csv
data_Skip_frame_completion/level_1/1_2.csv
data_Skip_frame_completion/level_1/1_3.csv
data_Skip_frame_completion/level_1/1_4.csv
data_Skip_frame_completion/level_1/1_5.csv
data_Skip_frame_completion/level_1/1_6.csv
data_Skip_frame_completion/level_1/1_7.csv
data_Skip_frame_completion/level_1/1_8.csv
data_Skip_frame_completion/level_1/1_9.csv
data_Skip_frame_completion/level_1/1_10.csv
data_Skip_frame_completion/level_1/1_11.csv
data_Skip_frame_completion/level_1/1_12.csv
data_Skip_frame_completion/level_1/1_13.csv
data_Skip_frame_completion/level_1/1_14.csv
data_Skip_frame_completion/level_1/1_15.csv
data_Skip_frame_completion/level_1/1_16.csv
data_Skip_frame_completion/level_1/1_17.csv
data_Skip_frame_completion/level_1/1_18.csv
data_Skip_frame_completion/level_1/1_19.csv
data_Skip_frame_completion/level_1/1_20.csv
data_Skip_frame_completion/level_1/1_21.csv
data_Skip_frame_completion/level_1/1_22.csv
data_Skip_frame_completion/level_1/1_23.c

data_Skip_frame_completion/level_2/2_98.csv
data_Skip_frame_completion/level_2/2_99.csv
data_Skip_frame_completion/level_2/2_100.csv
data_Skip_frame_completion/level_2/2_101.csv
data_Skip_frame_completion/level_2/2_102.csv
data_Skip_frame_completion/level_2/2_103.csv
data_Skip_frame_completion/level_2/2_104.csv
data_Skip_frame_completion/level_2/2_105.csv
data_Skip_frame_completion/level_2/2_106.csv
data_Skip_frame_completion/level_2/2_107.csv
data_Skip_frame_completion/level_2/2_108.csv
data_Skip_frame_completion/level_2/2_109.csv
data_Skip_frame_completion/level_2/2_110.csv
data_Skip_frame_completion/level_2/2_111.csv
data_Skip_frame_completion/level_2/2_112.csv
data_Skip_frame_completion/level_2/2_113.csv
data_Skip_frame_completion/level_2/2_114.csv
data_Skip_frame_completion/level_2/2_115.csv
data_Skip_frame_completion/level_2/2_116.csv
data_Skip_frame_completion/level_2/2_117.csv
data_Skip_frame_completion/level_2/2_118.csv
data_Skip_frame_completion/level_2/2_119.csv
data_Skip_fr

In [3]:
import pandas as pd
import numpy as np
import math
from warnings import simplefilter

simplefilter(action='ignore') #忽略警告



for level in range(0,3):
    if level == 0 :
        index_max = 199
    if level == 1 :
        index_max = 90
    if level == 2 :
        index_max = 132
    for index in range(1,index_max+1):
        file_name="data_Skip_frame_completion/level_"+str(level)+"/"+str(level)+"_"+str(index)+".csv";
        print(file_name)
        data = pd.read_csv(file_name,encoding = 'gb2312',names=['id', 'frame', 'x', 'y'])  # 读入csv文件中的特征
        #眼部
        #求最大值
        #定义几个要用的变量
        left_max=0;
        right_max=0;
        length=data.shape[0];
        for i in range(0,length-1):
            i=i+1;
            id=data.iloc[i]["id"];
            if id == "39":
                x39=float(data.iloc[i]["x"])
                x36=float(data.iloc[i-3]["x"])
                x45=float(data.iloc[i+6]["x"])
                x42=float(data.iloc[i+3]["x"])
                w_left_sub=x39-x36;#左眼的宽度差值
                w_right_sub=x45-x42;#右眼的宽度差值
                y38=float(data.iloc[i-1]["y"])
                y40=float(data.iloc[i+1]["y"])
                y43=float(data.iloc[i+4]["y"])
                y47=float(data.iloc[i+8]["y"])
                h_left_sub=y40-y38;#左眼的高度差值
                h_right_sub=y47-y43;#右眼的高度差值

                #求最大值
                left_max_temp=w_left_sub*h_left_sub;
                right_max_temp=w_right_sub*h_right_sub;
                if left_max_temp>left_max:
                    left_max=left_max_temp;
                if right_max_temp>right_max:
                    right_max=right_max_temp;

#         print(left_max)                    
        length=data.shape[0];

        left_max_ratio={};
        right_max_ratio={};
        eyes_ratio={};

        K={};
        MOA={};
        center={};
        
        for i in range(0,length-1):
            i=i+1;
            id=data.iloc[i]["id"];
            frame=int(data.iloc[i]["frame"]);
            
            if id == "30" :
                center[frame]=float(data.iloc[i-2]["y"])
                
            if id == "39" :
                x39=float(data.iloc[i]["x"])
                x36=float(data.iloc[i-3]["x"])
                x45=float(data.iloc[i+6]["x"])
                x42=float(data.iloc[i+3]["x"])
                w_left_sub=x39-x36;#左眼的宽度差值
                w_right_sub=x45-x42;#右眼的宽度差值
                y38=float(data.iloc[i-1]["y"])
                y40=float(data.iloc[i+1]["y"])
                y43=float(data.iloc[i+4]["y"])
                y47=float(data.iloc[i+8]["y"])
                h_left_sub=y40-y38;#左眼的高度差值
                h_right_sub=y47-y43;#右眼的高度差值

                #求百分比
                left_max_temp=w_left_sub*h_left_sub;
                left_max_ratio[frame]=left_max_temp/left_max;#左眼目前/左眼最大
                right_max_temp=w_right_sub*h_right_sub;
                right_max_ratio[frame]=right_max_temp/right_max;
                eyes_ratio[frame]=(left_max_ratio[frame]+right_max_ratio[frame])/2;

            if id == "60" :
                x60=float(data.iloc[i]["x"])
                y60=float(data.iloc[i]["y"])
                x62=float(data.iloc[i+2]["x"])
                y62=float(data.iloc[i+2]["y"])
                x66=float(data.iloc[i+6]["x"])
                y66=float(data.iloc[i+6]["y"])
                #点62到点60距离的平方
                d_62_60_2=pow((x62-x60), 2)+ pow((y62-y60), 2)
                d_66_60_2=pow((x66-x60), 2)+ pow((y66-y60), 2)
                d_66_62_2=pow((x66-x62), 2)+ pow((y66-y62), 2)

                if d_62_60_2 == 0:
                    d_62_60_2 = 1
                if d_66_60_2 == 0:
                    d_66_60_2 = 1
                #距离
                d_62_60=pow(d_62_60_2, 0.5)
                d_66_60=pow(d_66_60_2, 0.5)
                d_66_62=pow(d_66_62_2, 0.5)
                MOA[frame]=(d_62_60_2 + d_66_60_2 - d_66_62_2) / (2 * d_62_60 * d_66_60)
#                 print("MOA是",MOA[frame])


            #输出到文件

        #字典中的key值即为csv中的列名

        frame_col={};
            
        for k in center:
            frame_col[k]=k;

        df = pd.DataFrame({'frame':frame_col,'center':center,
                          'eyes_ratio':eyes_ratio,
                          'MOA':MOA})
 
        file_name_new="data_new_2/level_"+str(level)+"/"+str(level)+"_"+str(index)+".csv";
        df.to_csv(file_name_new,index=False,sep=',')

data_Skip_frame_completion/level_0/0_1.csv
data_Skip_frame_completion/level_0/0_2.csv
data_Skip_frame_completion/level_0/0_3.csv
data_Skip_frame_completion/level_0/0_4.csv
data_Skip_frame_completion/level_0/0_5.csv
data_Skip_frame_completion/level_0/0_6.csv
data_Skip_frame_completion/level_0/0_7.csv
data_Skip_frame_completion/level_0/0_8.csv
data_Skip_frame_completion/level_0/0_9.csv
data_Skip_frame_completion/level_0/0_10.csv
data_Skip_frame_completion/level_0/0_11.csv
data_Skip_frame_completion/level_0/0_12.csv
data_Skip_frame_completion/level_0/0_13.csv
data_Skip_frame_completion/level_0/0_14.csv
data_Skip_frame_completion/level_0/0_15.csv
data_Skip_frame_completion/level_0/0_16.csv
data_Skip_frame_completion/level_0/0_17.csv
data_Skip_frame_completion/level_0/0_18.csv
data_Skip_frame_completion/level_0/0_19.csv
data_Skip_frame_completion/level_0/0_20.csv
data_Skip_frame_completion/level_0/0_21.csv
data_Skip_frame_completion/level_0/0_22.csv
data_Skip_frame_completion/level_0/0_23.c

data_Skip_frame_completion/level_0/0_186.csv
data_Skip_frame_completion/level_0/0_187.csv
data_Skip_frame_completion/level_0/0_188.csv
data_Skip_frame_completion/level_0/0_189.csv
data_Skip_frame_completion/level_0/0_190.csv
data_Skip_frame_completion/level_0/0_191.csv
data_Skip_frame_completion/level_0/0_192.csv
data_Skip_frame_completion/level_0/0_193.csv
data_Skip_frame_completion/level_0/0_194.csv
data_Skip_frame_completion/level_0/0_195.csv
data_Skip_frame_completion/level_0/0_196.csv
data_Skip_frame_completion/level_0/0_197.csv
data_Skip_frame_completion/level_0/0_198.csv
data_Skip_frame_completion/level_0/0_199.csv
data_Skip_frame_completion/level_1/1_1.csv
data_Skip_frame_completion/level_1/1_2.csv
data_Skip_frame_completion/level_1/1_3.csv
data_Skip_frame_completion/level_1/1_4.csv
data_Skip_frame_completion/level_1/1_5.csv
data_Skip_frame_completion/level_1/1_6.csv
data_Skip_frame_completion/level_1/1_7.csv
data_Skip_frame_completion/level_1/1_8.csv
data_Skip_frame_completion

data_Skip_frame_completion/level_2/2_84.csv
data_Skip_frame_completion/level_2/2_85.csv
data_Skip_frame_completion/level_2/2_86.csv
data_Skip_frame_completion/level_2/2_87.csv
data_Skip_frame_completion/level_2/2_88.csv
data_Skip_frame_completion/level_2/2_89.csv
data_Skip_frame_completion/level_2/2_90.csv
data_Skip_frame_completion/level_2/2_91.csv
data_Skip_frame_completion/level_2/2_92.csv
data_Skip_frame_completion/level_2/2_93.csv
data_Skip_frame_completion/level_2/2_94.csv
data_Skip_frame_completion/level_2/2_95.csv
data_Skip_frame_completion/level_2/2_96.csv
data_Skip_frame_completion/level_2/2_97.csv
data_Skip_frame_completion/level_2/2_98.csv
data_Skip_frame_completion/level_2/2_99.csv
data_Skip_frame_completion/level_2/2_100.csv
data_Skip_frame_completion/level_2/2_101.csv
data_Skip_frame_completion/level_2/2_102.csv
data_Skip_frame_completion/level_2/2_103.csv
data_Skip_frame_completion/level_2/2_104.csv
data_Skip_frame_completion/level_2/2_105.csv
data_Skip_frame_completion